# Lab 18: Fine-Tuning for Security

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab18_fine_tuning.ipynb)

Build custom security-focused AI models through fine-tuning.

## Learning Objectives
- Custom embedding training
- LoRA fine-tuning concepts
- Security-specific evaluation
- Model deployment strategies

In [ ]:
!pip install sentence-transformers scikit-learn pandas -q

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict

## 1. Security Text Embeddings

Standard embeddings may not capture security-specific semantics.

In [ ]:
from sentence_transformers import SentenceTransformer

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Security-related texts
security_texts = [
    "PowerShell encoded command execution detected",
    "Base64 encoded PowerShell payload found",
    "User logged in from new location",
    "Failed SSH authentication attempt",
    "Brute force password attack detected",
]

# Get embeddings
embeddings = model.encode(security_texts)

# Calculate similarity matrix
similarity_matrix = cosine_similarity(embeddings)

print("Similarity Matrix:")
df_sim = pd.DataFrame(similarity_matrix, 
                       index=[f"T{i}" for i in range(len(security_texts))],
                       columns=[f"T{i}" for i in range(len(security_texts))])
print(df_sim.round(2))

print("\nTexts:")
for i, text in enumerate(security_texts):
    print(f"T{i}: {text[:50]}...")

## 2. Creating Training Data for Fine-Tuning

In [ ]:
# Security-specific training pairs
# Format: (anchor, positive, negative)
TRAINING_TRIPLETS = [
    # Similar attack types should be close
    ("PowerShell encoded command execution", 
     "Base64 encoded PowerShell payload",
     "User login from new device"),
    
    ("Failed SSH brute force attempt",
     "Multiple failed RDP authentication",
     "Scheduled task created"),
    
    ("Lateral movement via PsExec",
     "Remote execution using WMI",
     "File download completed"),
    
    ("Credential dumping from LSASS",
     "Mimikatz execution detected",
     "Normal user logout"),
]

print(f"Training triplets: {len(TRAINING_TRIPLETS)}")
print("\nExample triplet:")
print(f"  Anchor: {TRAINING_TRIPLETS[0][0]}")
print(f"  Positive: {TRAINING_TRIPLETS[0][1]}")
print(f"  Negative: {TRAINING_TRIPLETS[0][2]}")

## 3. Fine-Tuning Process (Conceptual)

In [ ]:
# This is a conceptual example - actual fine-tuning requires more data and compute

from sentence_transformers import losses, InputExample
from torch.utils.data import DataLoader

def prepare_training_data(triplets: List[tuple]) -> List[InputExample]:
    """Convert triplets to training examples."""
    examples = []
    for anchor, positive, negative in triplets:
        examples.append(InputExample(texts=[anchor, positive, negative]))
    return examples

# Prepare data
train_examples = prepare_training_data(TRAINING_TRIPLETS)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

# Define loss function
train_loss = losses.TripletLoss(model=model)

print("Fine-tuning configuration:")
print(f"  Training examples: {len(train_examples)}")
print(f"  Loss function: TripletLoss")
print(f"  Model: {model.__class__.__name__}")

# NOTE: Actual training would be:
# model.fit(train_objectives=[(train_dataloader, train_loss)],
#           epochs=10,
#           warmup_steps=100)

## 4. LoRA Fine-Tuning Concepts

In [ ]:
# LoRA (Low-Rank Adaptation) concept visualization

def explain_lora():
    """
    LoRA adds small trainable matrices to frozen base model.
    
    Original: W (frozen, large matrix)
    LoRA: W + BA where B and A are small matrices
    
    Benefits:
    - Much fewer parameters to train
    - Can swap adapters for different tasks
    - Base model stays frozen (stable)
    """
    
    # Simulate parameter counts
    base_params = 125_000_000  # 125M parameter model
    rank = 8  # LoRA rank
    hidden_dim = 768
    n_layers = 12
    
    lora_params = 2 * rank * hidden_dim * n_layers * 2  # A and B matrices, Q and V
    
    print("LoRA Efficiency:")
    print(f"  Base model parameters: {base_params:,}")
    print(f"  LoRA parameters: {lora_params:,}")
    print(f"  Reduction: {lora_params/base_params:.4%}")

explain_lora()

## 5. Security-Specific Evaluation

In [ ]:
# Security benchmark evaluation

SECURITY_BENCHMARK = [
    # (query, expected_similar, expected_different)
    ("Credential theft via Mimikatz", "LSASS memory dump", "User password change"),
    ("C2 beacon over HTTPS", "Encrypted C2 traffic", "Normal web browsing"),
    ("Ransomware encryption detected", "Files encrypted with high entropy", "Backup job completed"),
]

def evaluate_security_model(model, benchmark: List[tuple]) -> Dict:
    """Evaluate model on security benchmark."""
    correct = 0
    total = len(benchmark)
    
    for query, similar, different in benchmark:
        embeddings = model.encode([query, similar, different])
        
        # Calculate similarities
        sim_similar = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
        sim_different = cosine_similarity([embeddings[0]], [embeddings[2]])[0][0]
        
        # Check if similar is closer than different
        if sim_similar > sim_different:
            correct += 1
    
    return {
        "accuracy": correct / total,
        "correct": correct,
        "total": total
    }

results = evaluate_security_model(model, SECURITY_BENCHMARK)
print(f"Security Benchmark Results:")
print(f"  Accuracy: {results['accuracy']:.2%}")
print(f"  Correct: {results['correct']}/{results['total']}")

## 6. Deployment Considerations

In [ ]:
# Model deployment configuration

DEPLOYMENT_CONFIG = {
    "model_name": "security-embeddings-v1",
    "base_model": "all-MiniLM-L6-v2",
    "fine_tuning": {
        "method": "triplet_loss",
        "epochs": 10,
        "batch_size": 32
    },
    "deployment": {
        "format": "ONNX",
        "quantization": "int8",
        "max_batch_size": 64,
        "max_sequence_length": 256
    },
    "monitoring": {
        "track_latency": True,
        "track_drift": True,
        "alert_threshold": 0.1
    }
}

import json
print("Deployment Configuration:")
print(json.dumps(DEPLOYMENT_CONFIG, indent=2))

## Key Takeaways

1. **Domain Embeddings**: Security text needs specialized embeddings
2. **Triplet Loss**: Train similar items to be close, different items far
3. **LoRA**: Efficient fine-tuning with minimal parameters
4. **Evaluation**: Use security-specific benchmarks

## Next Steps
- **Lab 19**: Cloud Security AI
- Apply fine-tuned models to your security use cases